- load data
- feature vectors: extract from pictures with pre trained with Neural Network MobileNetV2 or ResNet50
=> here it will be flattend by the cnn
- 

### 0. Import packages and load the data

In [1]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tqdm import tqdm


df = pd.read_excel("/Users/felix/Documents/Data Science/05_The_project_plant_recognition/Data-Scientist-Project-Plant-Recognition-/02_analysis/01_computed_data_sets_plant_dataset_detailed.xlsx")

In [2]:

# === CONFIG ===
IMAGE_SIZE = (224, 224)
DATASET_PATH = "/Users/felix/Documents/Data Science/05_The_project_plant_recognition/Data-Scientist-Project-Plant-Recognition-/01_data/2.1.1 New Plant Diseases"
FEATURES_OUTPUT = "features.npy"
LABELS_OUTPUT = "labels.npy"

# === LOAD CNN (pre-trained, feature extractor only) ===
model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

# === HELPER: Preprocess image ===
def preprocess_image(img_path):
    try:
        img = image.load_img(img_path, target_size=IMAGE_SIZE)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        return preprocess_input(img_array)
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None

# === FEATURE EXTRACTION ===
features = []
labels = []

for label in tqdm(sorted(os.listdir(DATASET_PATH)), desc="Processing classes"):
    class_dir = os.path.join(DATASET_PATH, label)
    if not os.path.isdir(class_dir):
        continue

    for fname in os.listdir(class_dir):
        fpath = os.path.join(class_dir, fname)
        if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        pre_img = preprocess_image(fpath)
        if pre_img is None:
            continue

        feature = model.predict(pre_img, verbose=0).flatten()
        features.append(feature)
        labels.append(label)

# === SAVE TO DISK ===
features = np.array(features)
labels = np.array(labels)

np.save(FEATURES_OUTPUT, features)
np.save(LABELS_OUTPUT, labels)

print(f"\n✅ Done. Extracted {features.shape[0]} feature vectors of dimension {features.shape[1]}")


/var/folders/k_/x6px3sls6sl0yvv6s6pgyqh80000gn/T/ipykernel_53867/1067779979.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")
Processing classes: 100%|██████████| 3/3 [00:00<00:00, 2851.97it/s]


IndexError: tuple index out of range